## Function Calling in OpenAI

In [ ]:
#!pip install openai
#!pip install python-dotenv

In [1]:
import json
import pandas as pd
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

In [2]:
def calculateLoanPayment(principal: float, interestRate: float, term: int) -> float:
    # Convert annual interest rate to a monthly rate
    monthly_rate = interestRate / 100 / 12
    # Total number of payments (months)
    total_payments = term * 12
    
    # Calculate monthly payment using the loan payment formula
    if monthly_rate == 0:
        # Avoid division by zero in case of 0% interest
        monthly_payment = principal / total_payments
    else:
        monthly_payment = (principal * monthly_rate * (1 + monthly_rate) ** total_payments) / ((1 + monthly_rate) ** total_payments - 1)
    
    return monthly_payment



In [ ]:
principal = 50000  
interestRate = 5
term = 15  

monthly_payment = calculateLoanPayment(principal, interestRate, term)
print(f"The monthly payment is: ${monthly_payment:.2f}")

In [ ]:
def calculateMonthlySavings(goal_amount: float, interest_rate: float, time_in_years: int) -> float:
    # Convert annual interest rate to a monthly rate
    monthly_rate = interest_rate / 100 / 12
    # Total number of months
    total_months = time_in_years * 12

    if monthly_rate == 0:
        # No interest case
        monthly_savings = goal_amount / total_months
    else:
        # Calculate required monthly savings with interest
        monthly_savings = goal_amount * monthly_rate / ((1 + monthly_rate) ** total_months - 1)

    return monthly_savings

In [ ]:
goal_amount = 50000  
interestRate = 5
time_in_years = 15  

monthly_savings = calculateMonthlySavings(goal_amount, interestRate, time_in_years)
print(f"Your monthly savings should be: ${monthly_savings:.2f}")

In [ ]:
calc_parameters = {
    "type": "object",
    "properties": {
        "principal": {
            "type": "number",
            "description": "The loan amount or principal in dollars."
        },
        "interestRate": {
            "type": "number",
            "description": "The annual interest rate as a percentage (e.g., 5 for 5%)."
        },
        "term": {
            "type": "integer",
            "description": "The loan term in years."
        }
    },
    "required": ["principal", "interestRate", "term"]
}

calc_loan_tool = {
    "type": "function",
    "function": {
        "name": "calculateLoanPayment",
        "description": "Calculates loan payment from user inputs",
        "parameters": calc_parameters,
    }
}

In [ ]:
savings_goal_parameters = {
    "type": "object",
    "properties": {
        "goal_amount": {
            "type": "number",
            "description": "The total savings goal amount in dollars."
        },
        "interest_rate": {
            "type": "number",
            "description": "The annual interest rate as a percentage (e.g., 2 for 2%)."
        },
        "time_in_years": {
            "type": "integer",
            "description": "The time period in years to reach the savings goal."
        }
    },
    "required": ["goal_amount", "interest_rate", "time_in_years"]
}

savings_goal_tool = {
    "type": "function",
    "function": {
        "name": "calculateMonthlySavings",
        "description": "Calculates the required monthly savings to reach a specified goal within a set timeframe, factoring in interest.",
        "parameters": savings_goal_parameters
    }
}

In [ ]:
def get_output(messages_input, model_input="gpt-4o-mini", temperature_input = 0) :

    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
        tools = [calc_loan_tool, savings_goal_tool],
        tool_choice = "auto"
    )
    
    response = response.choices[0].message
    return response

In [ ]:
def initialize_bot():
    system_message = f"""
    You are a financial assistant chatbot here to help users explore their options for funding an upcoming purchase. 
    Begin by asking questions to understand the user's goal: specifically, whether they’d like to save for their purchase or take out a loan.

    You have access to two tools to assist with these options:

    calculateLoanPayment: Calculates the monthly payment if the user decides to borrow the amount needed.
    calculateMonthlySavings: Calculates the monthly savings needed to reach the goal by a specific deadline, 
    factoring in potential interest on the savings.

    Start by asking the user questions to clarify their goal and preferences. 
    If the user indicates they would like to borrow, gather details such as the loan amount, annual interest rate, and loan term in years,
    then use calculateLoanPayment to calculate the monthly payment. If the user prefers to save, ask about the target savings amount,
    the time frame, and expected interest rate, then use calculateMonthlySavings to estimate the monthly savings needed.

    Throughout the conversation, ensure the user understands the parameters and options.
    Provide explanations or suggestions on adjusting loan terms, rates, or the timeframe
    to align with their financial goals. 
    End each response by confirming if they need further assistance or if they’d like help exploring other options.
    """
    
    context = [{"role":"system", "content":system_message}]

    return context


In [ ]:
def collect_messages(prompt, context):
    context.append({'role':'user', 'content':prompt})
    response = get_output(context) 
    context.append({'role':'assistant', 'content': f"{response.content}"})
    return response

In [ ]:
def main():
    context = initialize_bot()

    print("Welcome! Type 'exit' to end the chat.")
    greeting = collect_messages("", context)
    print(greeting.content)
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'exit':
            return context
        
        response = collect_messages(user_input, context)

        # Display the assistant's response if it has content
        if response.content != None:
            print("Bot:", response.content)

        # Execute tool call(s) if any included in assistant's response
        if response.tool_calls:
            for tool_call in response.tool_calls:
                # Extract tool name and arguments from each tool call
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                
                # Handle the calculateLoanPayment tool call
                if tool_name == "calculateLoanPayment":
                    principal = arguments["principal"]
                    interestRate = arguments["interestRate"]
                    term = arguments["term"]
                    
                    # Call calculateLoanPayment and add the result to context
                    monthly_payment = calculateLoanPayment(principal, interestRate, term)
                    context.append({
                        'role': 'function', 'name': 'calculateLoanPayment','content': f"Calculated Monthly Payment: ${monthly_payment:.2f}"})
                
                # Handle the compareLoans tool call
                if tool_name == "calculateMonthlySavings":
                    goal_amount = arguments["goal_amount"]
                    interest_rate = arguments["interest_rate"]
                    time_in_years = arguments["time_in_years"]
    
                    # Call calculateMonthlySavings and add the result to context
                    monthly_savings = calculateMonthlySavings(goal_amount, interest_rate, time_in_years)
                    context.append({'role': 'function','name': 'calculateMonthlySavings', 'content': f"You need to save approximately ${monthly_savings:.2f} per month."})
                
                #After processing each tool call, continue the conversation with updated context
                follow_up_response = collect_messages("summarize results for customer", context)
                print("Bot:", follow_up_response.content)


        

In [ ]:
chat_transcript = main()

In [ ]:
chat_transcript